In [35]:
from Bimatrix import PayoffMatrix
import torch
import fractions

In [36]:
payoff = PayoffMatrix(None,10,5)

In [37]:
payoff.__str__()


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [38]:
payoff = PayoffMatrix([[1,0],[0,0]] , 2, 2)

In [39]:
torch.concat((payoff.negmatrix, torch.Tensor([[0,0]])))

tensor([[1., 2.],
        [2., 2.],
        [0., 0.]])